In [1]:
import os
print (os.getcwd())

C:\Users\user


# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


#----------------------------------
import pickle
import warnings
import string
import unittest
warnings.filterwarnings("ignore")
#----------------------------------


import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem         import WordNetLemmatizer

from nltk.corpus import stopwords

nltk.download(['punkt', 'wordnet','stopwords'])


# ------------------------------------------
from nltk.stem.porter import PorterStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
# ------------------------------------------



from sklearn.metrics                 import confusion_matrix
from sklearn.model_selection         import train_test_split
from sklearn.ensemble                import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput             import MultiOutputClassifier
from sklearn.utils                   import shuffle
from sklearn.datasets                import make_classification
from sklearn.pipeline                import Pipeline, FeatureUnion


# import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection         import train_test_split, GridSearchCV
from sklearn.metrics                 import precision_score, recall_score, f1_score,classification_report, make_scorer
from sklearn.ensemble                import AdaBoostClassifier
from sklearn.base                    import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
## Execute this code cell to output the values in the categories table
# connect to the database
engine = create_engine('sqlite:///disaster.db')
# the database file will be disaster.db
df = pd.read_sql_table('disaster', engine)
# load data from database
df = pd.read_sql_table('disaster', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)



### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens


In [5]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat =  confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1),labels=labels)
    
    #confusion_mat =  confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    #confusion_matrix(y_test , y_pred   , labels=labels)
                   
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def build_pipeline():
    pipeline = Pipeline ([
        ('vect'    , CountVectorizer(tokenizer=tokenize)),
        ('tfidf'   , TfidfTransformer()),
        ('clf'     , MultiOutputClassifier(RandomForestClassifier( ) ))     
    ])
    
    return pipeline      

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# train classifier    
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline = build_pipeline()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def display_results2(y_test, y_pred):

    results_dict = {}

    for pred, label, col in zip(y_pred.transpose(), y_test.values.transpose(), y_test.columns):
        print(col)
        print(classification_report(label, pred))
        results_dict[col] = classification_report(label, pred, output_dict=True)

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001FAF0A9D7B8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=Fal

In [14]:
#make_scorer(f1_score, average='micro')
#parameters = {
#'vect__max_df': [0.8]
#       ,'clf__estimator__max_depth': (25, 50, None)
#       ,'clf__estimator__min_samples_leaf': [1,5,8] 
#}

#cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1 ,verbose=10)


#parameters = {
#       'vect__max_df': [0.8]
#       ,'clf__estimator__max_depth': (25, 50, None)
#       ,'clf__estimator__max_features': ['log2', 'sqrt','auto']
#}

#,'clf__estimator__min_samples_split': (2, 10, 25, 50, 100)  
# ,'clf__estimator__min_samples_leaf': [1,5,8] 


In [14]:
make_scorer(f1_score, average='micro')
parameters = {
       'vect__max_df': [0.8]
       ,'clf__estimator__max_depth': (25, 50, None)
       ,'clf__estimator__min_samples_leaf': [1,5,8] 
       ,'clf__estimator__max_features': ['log2', 'sqrt','auto']
       ,'clf__estimator__min_samples_split': (2, 10, 25, 50, 100)     
}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1 ,verbose=10)


#parameters = {
#       'vect__max_df': [0.8]
#       ,'clf__estimator__max_depth': (25, 50, None)
#       ,'clf__estimator__max_features': ['log2', 'sqrt','auto']
#}

#,'clf__estimator__min_samples_split': (2, 10, 25, 50, 100)  
# ,'clf__estimator__min_samples_leaf': [1,5,8] 


In [15]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed: 11.9min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed: 14.2min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 15.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 15.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': [0.8], 'clf__estimator__max_depth': (25, 50, None), 'clf__estimator__min_samples_leaf': [1, 5, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [16]:
# predict on test data
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
#cv_results = model_performance(y_test, y_pred_cv)
display_results2(y_test, y_pred_cv)

related
              precision    recall  f1-score   support

           0       0.63      0.47      0.53      1571
           1       0.84      0.91      0.87      4928
           2       0.46      0.29      0.36        55

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.64      0.56      0.59      6554
weighted avg       0.78      0.80      0.79      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5417
           1       0.83      0.45      0.58      1137

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.86      0.71      0.76      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5917
           1       0.76      0.51      0.61       637

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.86      0.74      0.79      6554
weighted avg       0.93      0.94      0.93      6554

fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        77

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5932
           1       0.88      0.76      0.81       622

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.93      0.87      0.90      6554
weighted avg       0.97      0.97      0.97      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
import pickle
import sys
model_filepath = sys.argv[1:]

In [19]:
# export model to pickle file
model_filepath = sys.argv[1:]
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))

### 9. Export your model as a pickle file

In [20]:
pkl_filename = "classifier.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [8]:
statistics = df.describe()


In [9]:
statistics

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
import plotly.graph_objs as go
import plotly
from plotly import tools
import plotly.figure_factory as ff

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Bar



In [36]:
df2 = df.describe().T


In [40]:
table_cat = df.describe(include=['O']).T, index=True, index_title='Categorical columns')


In [41]:
table_cat

In [45]:
df2

,count,mean,std,min,25%,50%,75%,max
id,26216.0,15224.821330,8826.889140,2.0,7446.75,15662.5,22924.25,30265.0
related,26216.0,0.773650,0.435276,0.0,1.00,1.0,1.00,2.0
request,26216.0,0.170659,0.376218,0.0,0.00,0.0,0.00,1.0
offer,26216.0,0.004501,0.066940,0.0,0.00,0.0,0.00,1.0
aid_related,26216.0,0.414251,0.492602,0.0,0.00,0.0,1.00,1.0
medical_help,26216.0,0.079493,0.270513,0.0,0.00,0.0,0.00,1.0
medical_products,26216.0,0.050084,0.218122,0.0,0.00,0.0,0.00,1.0
search_and_rescue,26216.0,0.027617,0.163875,0.0,0.00,0.0,0.00,1.0
security,26216.0,0.017966,0.132831,0.0,0.00,0.0,0.00,1.0
military,26216.0,0.032804,0.178128,0.0,0.00,0.0,0.00,1.0


In [34]:
table_cat = ff.create_table(df.describe(include=['O']).T, index=True, index_title='Categorical columns')
iplot(table_cat)

In [35]:
table_cat

In [73]:
import json
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)

# Show distribution of different category
category = list(df.columns[4:])
category_counts = []
for column_name in category:
    category_counts.append(np.sum(df[column_name]))    
    
# extract data for top 5 categories
categories = df.iloc[:,4:]
categories_mean_top5 = categories.mean().sort_values(ascending=False).head(5)
categories_names_top5 = list(categories_mean_top5.index)

# extract data for tail 5 categories
categories = df.iloc[:,4:]
categories_mean_tail5 = categories.mean().sort_values(ascending=False).tail(5)
categories_names_tail5 = list(categories_mean_tail5.index)

# create visuals
graphs = [
        {
            'data': [
                Bar(
                    x=genre_names,
                    y=genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
                Bar(
                    x=categories_names_top5,
                    y=categories_mean_top5
                )
            ],

            'layout': {
                'title': 'Top five Categories',
                'yaxis': {
                    'title': "Mean"
                },
                'xaxis': {
                    'title': "Category"
                }
            }
        },
        {
            'data': [
                Bar(
                    x=categories_names_tail5,
                    y=categories_mean_tail5
                )
            ],

            'layout': {
                'title': 'Tail five Categories',
                'yaxis': {
                    'title': "Mean"
                },
                'xaxis': {
                    'title': "Categories"
                }
            }
        }
    ]

# encode plotly graphs in JSON
ids = ["graph-{}".format(i) for i, _ in enumerate(graphs)]
graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)